In [ ]:
!pip install tensorflow==1.13.1
!pip install keras==2.0.8
!pip install h5py==2.10.0
!pip install scikit-image==0.16.2

In [ ]:

from google.colab import output
!unzip /content/drone-dataset-uav.zip -d /content/
output.clear()

In [ ]:
!git clone https://github.com/matterport/Mask_RCNN.git

In [ ]:
cd Mask_RCNN


In [ ]:
!python setup.py install

In [ ]:
pip show mask-rcnn

In [ ]:
# split into train and test set
from os import listdir
from xml.etree import ElementTree
from numpy import zeros
from numpy import asarray
from mrcnn.utils import Dataset
from mrcnn.utils import extract_bboxes
import matplotlib.pyplot as plt

In [ ]:
import shutil
import os
path = os.path.join("/content/dataset_xml_format/dataset_xml_format/", "images")
os.mkdir(path)
path = os.path.join("/content/dataset_xml_format/dataset_xml_format/", "annotation")
os.mkdir(path)

In [ ]:
for i in os.listdir("/content/dataset_xml_format/dataset_xml_format/"):
  if i.split(".")[-1]=='png':
    shutil.move("/content/dataset_xml_format/dataset_xml_format/"+i, "/content/dataset_xml_format/dataset_xml_format/images/"+i)
  elif i.split(".")[-1]=='xml':  
    shutil.move("/content/dataset_xml_format/dataset_xml_format/"+i, "/content/dataset_xml_format/dataset_xml_format/annotation/"+i)  
  elif i.split(".")[-1]=='jpg':
    shutil.move("/content/dataset_xml_format/dataset_xml_format/"+i, "/content/dataset_xml_format/dataset_xml_format/images/"+i)
  elif i.split(".")[-1]=='JPG':
    shutil.move("/content/dataset_xml_format/dataset_xml_format/"+i, "/content/dataset_xml_format/dataset_xml_format/images/"+i)

In [ ]:
import os
print(len(os.listdir("/content/dataset_xml_format/dataset_xml_format/annotation")))
print(len(os.listdir("/content/dataset_xml_format/dataset_xml_format/images")))

In [ ]:
# class that defines and loads the kangaroo dataset
class DroneDataset(Dataset):
	# load the dataset definitions
	def load_dataset(self, dataset_dir, is_train=True):
		# define one class
		self.add_class("dataset", 1, "drone")
		# define data locations
		images_dir = dataset_dir + '/content/dataset_xml_format/dataset_xml_format/images/'
		annotations_dir = dataset_dir + '/content/dataset_xml_format/dataset_xml_format/annotation/'
		# find all images
		for filename in listdir(images_dir):
			# extract image id
			image_id = filename[:-4]
			# skip bad images
  
			'''
      if image_id in ['00090']:
				continue
			# skip all images after 150 if we are building the train set
			if is_train and int(image_id) >= 150:
				continue
			# skip all images before 150 if we are building the test/val set
			if not is_train and int(image_id) < 150:
				continue
      '''
      
      
			img_path = images_dir + filename
			ann_path = annotations_dir + image_id + '.xml'
			# add to dataset
			self.add_image('dataset', image_id=image_id, path=img_path, annotation=ann_path)
   
 
	# extract bounding boxes from an annotation file
	def extract_boxes(self, filename):
		# load and parse the file
		tree = ElementTree.parse(filename)
		# get the root of the document
		root = tree.getroot()
		# extract each bounding box
		boxes = list()
		for box in root.findall('.//bndbox'):
			xmin = int(box.find('xmin').text)
			ymin = int(box.find('ymin').text)
			xmax = int(box.find('xmax').text)
			ymax = int(box.find('ymax').text)
			coors = [xmin, ymin, xmax, ymax]
			boxes.append(coors)
		# extract image dimensions
		width = int(root.find('.//size/width').text)
		height = int(root.find('.//size/height').text)
		return boxes, width, height
 
	# load the masks for an image
	def load_mask(self, image_id):
		# get details of image
		info = self.image_info[image_id]
		# define box file location
		path = info['annotation']
		# load XML
		boxes, w, h = self.extract_boxes(path)
		# create one array for all masks, each on a different channel
		masks = zeros([h, w, len(boxes)], dtype='uint8')
		# create masks
		class_ids = list()
		for i in range(len(boxes)):
			box = boxes[i]
			row_s, row_e = box[1], box[3]
			col_s, col_e = box[0], box[2]
			masks[row_s:row_e, col_s:col_e, i] = 1
			class_ids.append(self.class_names.index('drone'))
		return masks, asarray(class_ids, dtype='int32')
 
	# load an image reference
	def image_reference(self, image_id):
		info = self.image_info[image_id]
		return info['path']

In [ ]:
# train set
train_set = DroneDataset()
train_set.load_dataset('', is_train=True)
train_set.prepare()
print('Train: %d' % len(train_set.image_ids))

# test/val set
test_set = DroneDataset()
test_set.load_dataset('', is_train=False)
test_set.prepare()
print('Test: %d' % len(test_set.image_ids))

In [ ]:
import matplotlib.pyplot as plt
image_id = 40
image = train_set.load_image(image_id)
print(image.shape)
plt.axis("off")
plt.imshow(image)
# load image mask
mask, class_ids = train_set.load_mask(image_id)
plt.imshow(mask[:, :, 0], cmap='gray', alpha=0.5)
plt.show()
print(mask.shape)

In [ ]:
# extract bounding boxes from the masks
from mrcnn.visualize import display_instances
import numpy as np
bbox =extract_bboxes(mask)
# display image with masks and bounding boxes
display_instances(image, bbox, mask, class_ids, train_set.class_names)